## 격자 관련 전처리
- 앞의 주석 부분 : 격자 데이터(N=2310)의 각 격자에 1시간 간격 시계열(T=8640)을 설정
- 사고데이터 병합하여 각 격자 * 시간에 대한 사고발생유무(accident) 변수 생성

In [1]:
import pandas as pd
import geopandas as gpd
from tqdm import tqdm
from datetime import datetime
import numpy as np

In [6]:
# area = gpd.GeoDataFrame.from_file('/Users/dangchan/Desktop/Github/IOT_hackathon/data/LSMD_ADM_SECT_UMD_11.shp', encoding='cp949') # 경로 수정
grid = gpd.GeoDataFrame.from_file('/Users/dangchan/Desktop/Github/IOT_hackathon/Seoul_Grid/grid.shp') # 경로 각자 수정

In [ ]:
# datetime series for each row(grid)
dt_range = pd.date_range(start="2021-01-01", end="2022-01-01", freq='H')[:-1] # from 21/01/01 to 21/12/31 by hour
for i in tqdm(range(len(grid))):
    grid.dt[i] = dt_range

In [ ]:
# Explode! (about 30s)
'''
Explode to grid(spatial) * temporal dataset(N=20244360) > about 460 MB
각 격자별로 존재하는 시계열을 펼치는 과정
'''
grid_new = grid.explode('dt')
grid_new = grid_new.iloc[:,[1,2]]

In [ ]:
grid_new.to_pickle('grid_new.pkl') # 파일 저장용

In [3]:
grid_new = pd.read_pickle('grid_new.pkl')

In [14]:
# Merge with accident data > grid_new
df_acc = gpd.read_file('accidentdata.gpkg') # 사고 데이터셋(gpkg file)
grid_new['accident'] = 0
pd.set_option('mode.chained_assignment',  None) # 경고 무시용

for i in tqdm(range(len(df_acc))):
    idx = np.where(grid.geometry.contains(df_acc.geometry[i]))[0][0] # 각 사고데이터셋이 어느 격자에 포함되는지 인덱스 리턴
    dt = pd.to_datetime(df_acc.datetime[i]) # 각 사고데이터의 사고일시
    grid_new.loc[(grid_new.index == idx) & (grid_new.dt == dt),'accident']= 1 # 사고에 해당하는 격자 * 일시의 accident(사고유무)를 1로 변경

grid_new.loc[grid_new.accident == 1].head(5) # 사고 발생한 데이터의 첫 5개 출력

100%|██████████| 445/445 [00:24<00:00, 17.82it/s]


,geometry,dt,accident
51,"POLYGON ((938535.249 1951165.520, 938535.249 1...",2021-07-01 22:00:00,1
68,"POLYGON ((939035.249 1951665.520, 939035.249 1...",2021-05-06 21:00:00,1
89,"POLYGON ((939535.249 1952165.520, 939535.249 1...",2021-11-25 19:00:00,1
93,"POLYGON ((939535.249 1954165.520, 939535.249 1...",2021-01-27 21:00:00,1
97,"POLYGON ((940035.249 1943165.520, 940035.249 1...",2021-02-08 15:00:00,1


In [ ]:
grid_new.to_file('accidentdata_withgrid.gpkg') # 파일로 저장 (상당히 오래걸려서 패스하는게 좋을듯)